# Weather Augmentation Visualizer Demo

Om Anavekar (oanaveka)<br>
Autonomous Driving S25<br>
3/2/2025<br>

### Libraries

In [ ]:
import sys
import os

# Add the project root (parent directory) to sys.path
sys.path.append(os.path.abspath('..'))

In [ ]:
# !pip install "imgaug==0.4.0" "imageio<3" "numpy<=1.23"

In [ ]:
import random
from PIL import Image
import numpy as np
from nuscenes.nuscenes import NuScenes
import imgaug.augmenters as iaa
import matplotlib.pyplot as plt

### Config

In [ ]:
config = {
    'version': 'v1.0-mini',
    'dataroot': '/scratch/nuscenes_v1-0_mini',
    'rain_prob': 0.5,
    'snow_prob': 0.3,
    'fog_prob': 0.2,
    'rain_intensity': 0.7,  # currently unused, retained for potential future control
    'snow_intensity': 0.5,
    'fog_intensity': 0.6,
    'clouds': True,
    'cloud_intensity': 0.3,
    'contrast_reduction': True,
    'contrast_factor': 0.8
}

In [ ]:
# Set random seet
random.seed(42)

In [ ]:
def get_weather_augmenters(cfg):
    aug_list = []

    if cfg['rain_prob'] > 0 and random.random() < cfg['rain_prob']:
        aug_list.append(iaa.Rain())

    if cfg['snow_prob'] > 0 and random.random() < cfg['snow_prob']:
        aug_list.append(iaa.Snowflakes())

    if cfg['fog_prob'] > 0 and random.random() < cfg['fog_prob']:
        # Fog intensity is simulated by repetition
        fog_repeats = int(1 + cfg['fog_intensity'] * 3)
        aug_list.append(iaa.Sequential([iaa.Fog()] * fog_repeats))

    if cfg.get('clouds', False):
        cloud_repeats = int(1 + cfg['cloud_intensity'] * 5)
        aug_list.append(iaa.Sequential([iaa.Clouds()] * cloud_repeats))

    if cfg.get('contrast_reduction', False):
        aug_list.append(iaa.LinearContrast(cfg['contrast_factor']))

    return iaa.Sequential(aug_list)


In [ ]:
# ---------------------- Show Utility -----------------------------------
def show_transformation_result(original, transformed):
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    axs[0].imshow(original)
    axs[0].set_title("Original Image")
    axs[0].axis("off")
    axs[1].imshow(transformed)
    axs[1].set_title("Augmented Image")
    axs[1].axis("off")
    plt.tight_layout()
    plt.show()

In [ ]:
# ---------------------- Main Script -------------------------------------
# Load nuScenes
nusc = NuScenes(version=config['version'], dataroot=config['dataroot'], verbose=True)

# Random sample + random camera
sample = nusc.sample[random.randint(0, len(nusc.sample) - 1)]
cam_list = ['CAM_FRONT', 'CAM_FRONT_RIGHT', 'CAM_BACK_RIGHT', 'CAM_BACK', 'CAM_BACK_LEFT', 'CAM_FRONT_LEFT']
cam_token = sample['data'][random.choice(cam_list)]

# Load image
img_path = nusc.get_sample_data_path(cam_token)
original_img = Image.open(img_path).convert('RGB')
original_np = np.array(original_img)

# Apply augmentation
weather_aug = get_weather_augmenters(config)
augmented_np = weather_aug(image=original_np)
augmented_img = Image.fromarray(augmented_np)

# Show result
show_transformation_result(original_np, augmented_np)